Загрузим библиотеку fpgen

In [1]:
from IPython.display import clear_output

! pip install --upgrade git+https://github.com/rimgro/biocadprotein.git

clear_output()

In [2]:
from fpgen.generator import ProteinGenerator
from fpgen.metrics import CustomMetric

from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein
from esm.utils.structure.protein_chain import ProteinChain

In [3]:
# GFP
gfp_protein = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb('1qy3', chain_id='A')
)
gfp_aplha_phelix = [5, 133, 9, 138, 11, 139, 13, 140, 15, 141, 142, 180, 147, 181, 24, 182, 26, 32, 33, 34, 162, 164, 37, 38, 39, 166, 41, 43, 50, 178, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 198, 202, 204, 79, 81, 82, 85, 215, 89, 217, 91, 92, 93, 219, 221, 222, 97, 103, 105, 109, 116, 118, 120, 122]

# mScarlet
mscarlet_protein = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb('5lk4', chain_id='A')
)
mscarlet_aplha_phelix = ...

# sfGFP
sfgfp_protein = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb('2b3p', chain_id='A')
)
sfgfp_aplha_phelix = ...

Загрузим из базы данных PBD белок GFP и создадим объект генератора

In [4]:
model = ESM3.from_pretrained('esm3-open').to('cuda')

gfp_generator = ProteinGenerator(gfp_protein, gfp_aplha_phelix, model)

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\esm\models\vqvae.py:286: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):  # type: ignore


В библиотеке уже есть заготовленная функция для подсчета RMSD по всей структуре белка. Но нет для подсчета по альфа-спирали (по техническим причинам). Поэтому ее необходимо реализовать самостоятельно

In [5]:
def phelix_rmsd(generation_protein, template_protein, alpha_phelix) -> float:
    template_chain: ProteinChain = template_protein.to_protein_chain()
    generation_chain: ProteinChain = generation_protein.to_protein_chain()

    rmsd = template_chain[alpha_phelix].rmsd(generation_chain[alpha_phelix])
    return rmsd

In [6]:
import os
os.environ['DISABLE_ITERATIVE_SAMPLING_TQDM'] = '1'

In [7]:
from tqdm import trange

In [8]:
metrics = []
proteins = []

In [9]:
protein, metric = gfp_generator.generate(
    metric_list=[
        CustomMetric(phelix_rmsd, gfp_aplha_phelix),
        'rmsd',
        'ptm',
        'plddt'
    ]
)

In [10]:
from fpgen import utils
fixed_protein = utils.fix_protein(protein)

1
2


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\esm\utils\structure\protein_complex.py:223: UserWarning: Entity ID not found in metadata, using None as default
  warnings.warn("Entity ID not found in metadata, using None as default")


3
4
5
6


In [ ]:
for _ in trange(1000):
    protein, metric = gfp_generator.generate(
        metric_list=[
            CustomMetric(phelix_rmsd, gfp_aplha_phelix),
            'rmsd',
            'ptm',
            'plddt'
        ]
    )

    metrics.append(metric)
    proteins.append(protein)

100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


In [ ]:
import pandas as pd
metrics_df = pd.DataFrame(metrics, columns=['alpha_rmsd', 'full_rmsd', 'ptm', 'plddt'])